In [ ]:
from flask import Flask, flash, redirect, render_template, request, session, abort
app = Flask(__name__)
 
@app.route("/")
def index():
    return "Index!"
 
'''@app.route("/findname/<string:name>/")
def helloMe(name):
    return render_template(
        'test.html',name=name) '''


@app.route("/index/")
def helloMe():
    return render_template(
        'index.html')
 
if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Apr/2018 20:46:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 20:46:57] "GET /index/ HTTP/1.1" 200 -
127.0.0.1 - - [17/Apr/2018 20:47:09] "GET /index/? HTTP/1.1" 200 -
